<a href="https://colab.research.google.com/github/JimmyCII/-Home_Sales/blob/main/Home_Sales_starter_code_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.5.5'
spark_version = 'spark-3.5.5'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Fetched 257 kB in 1s (210 kB/s)
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)


In [23]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [24]:
# 1. Read the home_sales_revised.csv from the provided AWS S3 bucket location into a PySpark DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"

spark.sparkContext.addFile(url)
home_sales_df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), header=True, inferSchema=True)
home_sales_df.head(10)

[Row(id='f8a53099-ba1c-47d6-9c31-7398aa8f6089', date=datetime.date(2022, 4, 8), date_built=2016, price=936923, bedrooms=4, bathrooms=3, sqft_living=3167, sqft_lot=11733, floors=2, waterfront=1, view=76),
 Row(id='7530a2d8-1ae3-4517-9f4a-befe060c4353', date=datetime.date(2021, 6, 13), date_built=2013, price=379628, bedrooms=2, bathrooms=2, sqft_living=2235, sqft_lot=14384, floors=1, waterfront=0, view=23),
 Row(id='43de979c-0bf0-4c9f-85ef-96dc27b258d5', date=datetime.date(2019, 4, 12), date_built=2014, price=417866, bedrooms=2, bathrooms=2, sqft_living=2127, sqft_lot=10575, floors=2, waterfront=0, view=0),
 Row(id='b672c137-b88c-48bf-9f18-d0a4ac62fb8b', date=datetime.date(2019, 10, 16), date_built=2016, price=239895, bedrooms=2, bathrooms=2, sqft_living=1631, sqft_lot=11149, floors=2, waterfront=0, view=0),
 Row(id='e0726d4d-d595-4074-8283-4139a54d0d63', date=datetime.date(2022, 1, 8), date_built=2017, price=424418, bedrooms=3, bathrooms=2, sqft_living=2249, sqft_lot=13878, floors=2, wa

In [25]:
# 2. Create a temporary view of the DataFrame.
home_sales_df.createOrReplaceTempView("home_sales")

print("Temporary view 'home_sales' created successfully!")

Temporary view 'home_sales' created successfully!


In [26]:
query = "SELECT * FROM home_sales LIMIT 5"
results = spark.sql(query)
results.show()

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
+--------------------+----------+----------+------+--------+---------+----------

In [27]:
# 3. What is the average price for a four bedroom house sold per year, rounded to two decimal places?
query_4bed = """
SELECT
    YEAR(date) AS sale_year,
    ROUND(AVG(price), 2) AS average_price
FROM
    home_sales
WHERE
    bedrooms = 4
GROUP BY
    sale_year
ORDER BY
    sale_year;
"""

avg_price_4bed_per_year = spark.sql(query_4bed)
avg_price_4bed_per_year.show()

+---------+-------------+
|sale_year|average_price|
+---------+-------------+
|     2019|     300263.7|
|     2020|    298353.78|
|     2021|    301819.44|
|     2022|    296363.88|
+---------+-------------+



In [28]:
# 4. What is the average price of a home for each year the home was built,
# that have 3 bedrooms and 3 bathrooms, rounded to two decimal places?
query_3bed_3bath = """
SELECT
    date_built,
    ROUND(AVG(price), 2) AS average_price
FROM
    home_sales
WHERE
    bedrooms = 3 AND bathrooms = 3
GROUP BY
    date_built
ORDER BY
    date_built;
"""

avg_price_3bed_3bath_per_year = spark.sql(query_3bed_3bath)
avg_price_3bed_3bath_per_year.show()

+----------+-------------+
|date_built|average_price|
+----------+-------------+
|      2010|    292859.62|
|      2011|    291117.47|
|      2012|    293683.19|
|      2013|    295962.27|
|      2014|    290852.27|
|      2015|     288770.3|
|      2016|    290555.07|
|      2017|    292676.79|
+----------+-------------+



In [29]:
# 5. What is the average price of a home for each year the home was built,
# that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet, rounded to two decimal places?
query_3bed_3bath_2floor_2000sqft = """
SELECT
    date_built,
    ROUND(AVG(price), 2) AS average_price
FROM
    home_sales
WHERE
    bedrooms = 3 AND bathrooms = 3 AND floors = 2 AND sqft_living >= 2000
GROUP BY
    date_built
ORDER BY
    date_built;
"""

avg_price_3bed_3bath_2floor_2000sqft_per_year = spark.sql(query_3bed_3bath_2floor_2000sqft)
avg_price_3bed_3bath_2floor_2000sqft_per_year.show()

+----------+-------------+
|date_built|average_price|
+----------+-------------+
|      2010|    285010.22|
|      2011|    276553.81|
|      2012|    307539.97|
|      2013|    303676.79|
|      2014|    298264.72|
|      2015|    297609.97|
|      2016|     293965.1|
|      2017|    280317.58|
+----------+-------------+



In [30]:
# 6. What is the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000? Order by descending view rating.
# Although this is a small dataset, determine the run time for this query.

start_time = time.time()

query_avg_price_per_view = """
SELECT
    view,
    ROUND(AVG(price), 2) AS average_price
FROM
    home_sales
GROUP BY
    view
HAVING
    AVG(price) >= 350000
ORDER BY
    view DESC;
"""

avg_price_per_view = spark.sql(query_avg_price_per_view)
avg_price_per_view.show()

end_time = time.time()
runtime = end_time - start_time

print("--- %s seconds ---" % (time.time() - start_time))

+----+-------------+
|view|average_price|
+----+-------------+
| 100|    1026669.5|
|  99|   1061201.42|
|  98|   1053739.33|
|  97|   1129040.15|
|  96|   1017815.92|
|  95|    1054325.6|
|  94|    1033536.2|
|  93|   1026006.06|
|  92|    970402.55|
|  91|   1137372.73|
|  90|   1062654.16|
|  89|   1107839.15|
|  88|   1031719.35|
|  87|    1072285.2|
|  86|   1070444.25|
|  85|   1056336.74|
|  84|   1117233.13|
|  83|   1033965.93|
|  82|    1063498.0|
|  81|   1053472.79|
+----+-------------+
only showing top 20 rows

--- 0.3619403839111328 seconds ---


In [31]:
# 7. Cache the the temporary table home_sales.
spark.catalog.cacheTable("home_sales")
print("Table 'home_sales' has been cached.")

Table 'home_sales' has been cached.


In [32]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [34]:
# 9. Using the cached data, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the uncached runtime.

start_time = time.time()

query_avg_price_per_view = """
SELECT
    view,
    ROUND(AVG(price), 2) AS average_price
FROM
    home_sales
GROUP BY
    view
HAVING
    AVG(price) >= 350000
ORDER BY
    view DESC;
"""

avg_price_per_view_cached = spark.sql(query_avg_price_per_view)
avg_price_per_view_cached.show()

end_time = time.time()
cached_runtime = end_time - start_time



print("--- %s seconds ---" % (time.time() - start_time))


+----+-------------+
|view|average_price|
+----+-------------+
| 100|    1026669.5|
|  99|   1061201.42|
|  98|   1053739.33|
|  97|   1129040.15|
|  96|   1017815.92|
|  95|    1054325.6|
|  94|    1033536.2|
|  93|   1026006.06|
|  92|    970402.55|
|  91|   1137372.73|
|  90|   1062654.16|
|  89|   1107839.15|
|  88|   1031719.35|
|  87|    1072285.2|
|  86|   1070444.25|
|  85|   1056336.74|
|  84|   1117233.13|
|  83|   1033965.93|
|  82|    1063498.0|
|  81|   1053472.79|
+----+-------------+
only showing top 20 rows

--- 0.43784046173095703 seconds ---


In [35]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
home_sales_df = home_sales_df.withColumn("date_built", home_sales_df["date_built"].cast("string"))
home_sales_df.write.partitionBy("date_built").parquet("home_sales_partitioned")
print("Data has been partitioned by 'date_built' and written to 'home_sales_partitioned' directory.")

Data has been partitioned by 'date_built' and written to 'home_sales_partitioned' directory.


In [36]:
# 11. Read the parquet formatted data.
partitioned_home_sales_df = spark.read.parquet("home_sales_partitioned")
partitioned_home_sales_df.printSchema()

root
 |-- id: string (nullable = true)
 |-- date: date (nullable = true)
 |-- price: integer (nullable = true)
 |-- bedrooms: integer (nullable = true)
 |-- bathrooms: integer (nullable = true)
 |-- sqft_living: integer (nullable = true)
 |-- sqft_lot: integer (nullable = true)
 |-- floors: integer (nullable = true)
 |-- waterfront: integer (nullable = true)
 |-- view: integer (nullable = true)
 |-- date_built: integer (nullable = true)



In [37]:
# 12. Create a temporary table for the parquet data.
partitioned_home_sales_df.createOrReplaceTempView("partitioned_home_sales")
print("Temporary view 'partitioned_home_sales' created successfully!")

Temporary view 'partitioned_home_sales' created successfully!


In [39]:
# 13. Using the parquet DataFrame, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the cached runtime.

start_time = time.time()

query = """
SELECT
    view,
    ROUND(AVG(price), 2) AS average_price
FROM
    partitioned_home_sales
GROUP BY
    view
HAVING
    AVG(price) >= 350000
ORDER BY
    view DESC;
"""

avg_price_per_view_partitioned = spark.sql(query)
avg_price_per_view_partitioned.show()

end_time = time.time()
partitioned_runtime = end_time - start_time

print(f"Query runtime (partitioned): {partitioned_runtime:.4f} seconds")

# print("--- %s seconds ---" % (time.time() - start_time))

+----+-------------+
|view|average_price|
+----+-------------+
| 100|    1026669.5|
|  99|   1061201.42|
|  98|   1053739.33|
|  97|   1129040.15|
|  96|   1017815.92|
|  95|    1054325.6|
|  94|    1033536.2|
|  93|   1026006.06|
|  92|    970402.55|
|  91|   1137372.73|
|  90|   1062654.16|
|  89|   1107839.15|
|  88|   1031719.35|
|  87|    1072285.2|
|  86|   1070444.25|
|  85|   1056336.74|
|  84|   1117233.13|
|  83|   1033965.93|
|  82|    1063498.0|
|  81|   1053472.79|
+----+-------------+
only showing top 20 rows

Query runtime (partitioned): 0.6781 seconds


In [40]:
# 14. Uncache the home_sales temporary table.
spark.catalog.uncacheTable("home_sales")
print("Table 'home_sales' has been uncached.")

Table 'home_sales' has been uncached.


In [41]:
# 15. Check if the home_sales is no longer cached
is_cached = spark.catalog.isCached("home_sales")
print(f"Is table 'home_sales' cached: {is_cached}")


Is table 'home_sales' cached: False
